## Revision datos IMFD

In [2]:
datos = open("datos/IMFD/grafo.ttl", "r")

declaracionesOld = 0
setDeclaraciones = set()
for i in datos:
    setDeclaraciones.add(i)
    declaracionesOld += 1
datos.close()
declaracionesOld

116412

In [3]:
len(setDeclaraciones)

42145

In [4]:
import re

In [5]:
topicoRegex = re.compile("<http://www.example.cl/topico/\d*>")
nombreTopicoRegex = re.compile('\".*\"')

In [6]:
dictTopicoNombre = {}

for i in setDeclaraciones:
    match = topicoRegex.match(i)
    if match != None:
        idTopico = int(match.group()[30:-1])
        match = nombreTopicoRegex.search(i)
        if match != None:
            dictTopicoNombre[idTopico] = match.group()[1:-1]

## Integracion

In [7]:
names = {
    1 : '"Derechos Fundamentales"',
    2 : '"Libertad De Asociacion"',
    3 : '"Derechos Civiles"',
    4 : '"Familia"',
    5 : '"Reformas y Constitucion"',
    6 : '"Reforma"',
    7 : '"Deber"',
    8 : '"Origen"',
    9 : '"Nacionalidad"',
    10: '"Deportacion"',
    11: '"Frontera"',
    12: '"Pueblos Originarios"',
    13: '"Religion"',
    14: '"Sistema Electoral"',
    15: '"Voto"',
    16: '"Eleccion"',
    17: '"Censo"',
    18: '"Distrito"',
    19: '"Partidos Politicos"',
    20: '"Partido"',
    21: '"Regulacion a Partidos"',
    22: '"Derecho a Patido"',
    23: '"Estado"',
    24: '"Ejecutivo"',
    25: '"Legislativo"',
    26: '"Territorio"',
    27: '"Judicial"',
    28: '"FFAA"',
    29: '"Derecho Internacional"',
    30: '"Economia"',
    31: '"Presupuesto"',
    32: '"Banco"',
    33: '"Expropiacion"',
    34: '"Mercado"',
    35: '"Propiedad"',
    36: '"Ocupacion"',
    39: '"Derechos Humanos"',
    42: '"Medios de Comunicacion"',
    43: '"Comunicacion"',
    44: '"Censura"',
    45: '"Reputacion"',
    46: '"Igualdad"',
    47: '"Color de Piel"',
    48: '"Estatus Social"',
    50: '"Orientacion Sexual"',
    51: '"Edad"',
    53: '"Marital"',
    54: '"Cultura"',
    55: '"Simbolos"',
    56: '"Ciencia"',
    57: '"Historia"',
    58: '"Derechos Sociales"',
    59: '"Educacion"',
    60: '"Medio Ambiente"',
    62: '"Desempleo"',
    66: '"Agua"',
    67: '"Trabajo"'
}

In [8]:
prefijos = {
    "topico" : "<http://www.example.cl/topico/",
    "type" : "<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>",
    "Topico" : "<http://www.imfd.cl/ontologia/convencion#Topico>",
    "nombre" : "<http://www.imfd.cl/ontologia/convencion#nombreTopico>"
}

In [9]:
formato = "{0} {1} {2} .\n"

In [10]:
setDeclaracionesNew = set()

for i in setDeclaraciones:
    match = topicoRegex.match(i)
    search = topicoRegex.search(i)
    if match != None:
        idTopico = int(match.group()[30:-1])
        if idTopico in names:
            if "type" in i:
                sujeto = prefijos["topico"] + dictTopicoNombre[idTopico] + ">"
                predicado = prefijos["type"]
                objeto = prefijos["Topico"]
                setDeclaracionesNew.add(formato.format(sujeto, predicado,objeto))
            
            if "nombreTopico" in i:
                sujeto = prefijos["topico"] + dictTopicoNombre[idTopico] + ">"
                predicado = prefijos["nombre"]
                objeto = names[idTopico]
                setDeclaracionesNew.add(formato.format(sujeto, predicado,objeto))
    
    elif search != None:
        idTopico = int(search.group()[30:-1])
        if idTopico in names:
            declaracion = i[:search.span()[0]] + prefijos["topico"] + dictTopicoNombre[idTopico] + "> .\n"
            setDeclaracionesNew.add(declaracion)
    else:
        setDeclaracionesNew.add(i)

In [11]:
datosNew = open("Ttl/grafo.ttl","w")
for i in setDeclaracionesNew:
    datosNew.write(i)
    
datosNew.close()   

## Integracion

In [12]:
import requests
url = "https://www.constituteproject.org/service/topics?lang=es"

# A GET request to the API
response = requests.get(url)

# Print the response
response_json = response.json()

In [13]:
nombreClases = {}
for topics in response_json:
    for subtopics in topics["topics"]:
        if topics["key"] not in nombreClases:
            nombreClases[topics["key"]] = []
        nombreClases[topics["key"]].append(subtopics["key"])
        for subsubtopics in subtopics["topics"]:
            if subtopics["key"] not in nombreClases:
                nombreClases[subtopics["key"]] = []
            nombreClases[subtopics["key"]].append(subsubtopics["key"])
            nombreClases[topics["key"]].append(subsubtopics["key"])

In [14]:
dictIntegracionA = {
    'derecho_fundamental': ["Civil_and_Political_Rights","assoc"],
    'libertad_asociacion': ["assoc"],
    'derecho_civil': ["Civil_and_Political_Rights"],
    'pueblo_originario': ["Indigenous_Groups"],
    'religion': ["Religion"],
    'voto': ["Suffrage_and_Turnout"],
    'eleccion': ["elections"],
    'censo': ["census"],
    'distrito': ["district"],
    'estado': ["executive","judiciary","legislature"],
    'ejecutivo': ["executive"],
    'legislativo': ["legislature"],
    'judicial': ["judiciary"],
    'ff_aa': ["Military"],
    'derecho_internacional': ["intlaw"],
    'economia': ["bank"],
    'dd_hh': ["hr"],
    'igualdad': ["equalgr10", "equalgr9", "equalgr8", "equalgr12", "equalgr16", "equalgr13", "equalgr7", "matequal"],
    'color_de_piel': ["equalgr10"],
    'estatus_social': ["equalgr12"],
    'orientacion_sexual': ["equalgr7"],
    'edad': ["equalgr8"],
    'marital': ["matequal"],
    'cultura': ["science", "prevcond"],
    'ciencia': ["science"],
    'historia': ["prevcond"],
    'derecho_social': ["water", "health"],
    'agua': ["water"],
}

In [15]:
dictIntegracionB = {
    'derecho_fundamental': ["fndfam"],
    'familia': ["fndfam"],
    'reforma_y_constitucion': ["abide", "amendment"],
    'reforma': ["amendment"],
    'deberes': ["abide"],
    'nacionalidad': ["natcit", "citizen"],
    'deportacion': ["citdep"],
    'frontera': ["resenex"],
    'partido': ["partreg"],
    'estado': ["Secession_and_Accession"],
    'territorio': ["Secession_and_Accession"],
    'ff_aa': ["Military"],
    'derecho_internacional': ["intlaw"],
    'economia': ["exprop", "freecomp", "occupate", "balbudgt", "budget", "proprght", "transfer", "intprop"],
    'presupuesto': ["balbudgt", "budget"],
    'expropiacion': ["exprop"],
    'mercado': ["freecomp"],
    'propiedad': ["proprght", "transfer", "intprop"],
    'ocupacion': ["occupate"],
    'medios_de_comunicacion': ["medcom", "telecom"],
    'comunicacion': ["medcom", "telecom"],
    'censura': ["medcom", "telecom"],
    'cultura': ["anthem", "flag"],
    'simbolos': ["anthem", "flag"],
    'derecho_social': ["finsup3", "finsup1", "achighed", "edfree", "edcomp", "env", "finsup4", "childwrk", "childpro", "provwork", "safework", "strike", "jointrde"],
    'educacion': ["achighed", "edfree", "edcomp"],
    'medio_ambiente': ["env"],
    'trabajo': ["provwork", "safework", "strike", "jointrde"],
}

In [16]:
prefijos2 = {
    "cc" : "<http://www.example.cl/topico/",
    "ccp": "<http://www.constituteproject.org/ontology/"
}

predicadoA = "<http://www.imfd.cl/ontologia/convencion#incluidoEn>"
predicadoB = "<http://www.imfd.cl/ontologia/convencion#incluyeParteDe>"

predicadoType = "<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>"

evento = re.compile("<http://www.constituteproject.org/constitution/.*_\d*>")



In [17]:
clasesEncotradas = set()

def buscarReferencia(dictIntegracion, declaracion):
    
    sujetos = set()
    
    claseCCP = re.compile("<http://www.constituteproject.org/ontology/.*>")
    search = claseCCP.search(declaracion)
    
    if search != None:
        clase = search.group()[43:-1]
        clasesEncotradas.add(clase)

        for topico in dictIntegracion:
            if clase in dictIntegracion[topico]:
                sujetos.add(prefijos2["cc"] + topico + ">")
                
            for claseCCP in dictIntegracion[topico]:
                if claseCCP in nombreClases:
                    if clase in nombreClases[claseCCP]:
                        sujetos.add(prefijos2["cc"] + topico + ">")
                
    return sujetos          
        

In [18]:
datos = open("Ttl/datosDuros.ttl", "r")
setIntegracion = set()

for declaracion in datos:
    if predicadoType in declaracion:
        sujetos = buscarReferencia(dictIntegracionA, declaracion)
        if len(sujetos) != 0:
            for sujeto in sujetos:
                search = evento.search(declaracion)
                objeto = search.group()
                declaracion = formato.format(sujeto, predicadoA, objeto)
                setIntegracion.add(declaracion)
            
        sujetos = buscarReferencia(dictIntegracionB, declaracion)
        if len(sujetos) != 0:
            for sujeto in sujetos:
                search = evento.search(declaracion)
                objeto = search.group()
                declaracion = formato.format(objeto, predicadoB, sujeto)
                setIntegracion.add(declaracion)
                    
datos.close()


In [19]:
integracion = open("Ttl/integracion.ttl", "w")

for declaracion in setIntegracion:
    integracion.write(declaracion)

integracion.close()